# Featurestore - Mars21
## Feature Store Setup
Christos Aniftos \
Soeren Petersen

## Install required libraries

## Import modules

In [ ]:
from google.cloud import aiplatform

In [ ]:
from google.api_core import operations_v1
from google.cloud.aiplatform_v1beta1.types import io as io_pb2
from google.cloud.aiplatform_v1beta1.types import FeaturestoreMonitoringConfig
from google.cloud.aiplatform_v1beta1.types.feature import Feature
from google.cloud.aiplatform_v1beta1 import FeaturestoreServiceClient
from google.cloud.aiplatform_v1beta1 import FeaturestoreOnlineServingServiceClient
from google.cloud.aiplatform_v1beta1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1beta1.types import feature_selector as feature_selector_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1beta1.types import featurestore_online_service as featurestore_online_service_pb2

In [ ]:
PROJECT_ID = "PROJECT_ID" # Change to your project id
LOCATION = "us-central1" 
API_ENDPOINT = LOCATION+"-aiplatform.googleapis.com" 
FEATURESTORE_ID = "universe"
ENTITY="planets"

## Define clinets for FS admin and data management

In [ ]:
# Create admin_client for CRUD
admin_client = FeaturestoreServiceClient(
    client_options={"api_endpoint": API_ENDPOINT})

In [ ]:
LOC_PATH = admin_client.common_location_path(PROJECT_ID, LOCATION)
FS_PATH = admin_client.featurestore_path(PROJECT_ID, LOCATION, FEATURESTORE_ID)
ENTITY_PATH = admin_client.entity_type_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY)
FEATURE_PATH = admin_client.feature_path(PROJECT_ID, LOCATION, FEATURESTORE_ID, ENTITY, '{}')

print("Location: \t", LOC_PATH)
print("Feature Store: \t", FS_PATH)
print("Entity: \t", ENTITY_PATH)
print("Feature: \t",FEATURE_PATH)

In [ ]:
# Create operation client to poll LRO status.
#lro_client = operations_v1.OperationsClient(admin_client.transport.grpc_channel)

## Create FeatureStore

In [ ]:
new_feature_store = featurestore_pb2.Featurestore(
            online_serving_config=featurestore_pb2.Featurestore.OnlineServingConfig(
                fixed_node_count=1, # The number of nodes for each cluster (bigtable)
            )
        )
try:
    print(
        admin_client.create_featurestore(
            featurestore_service_pb2.CreateFeaturestoreRequest(
                parent=LOC_PATH,
                featurestore_id=FEATURESTORE_ID,
                featurestore= new_feature_store
            )
        ).result()
    )
except Exception as ex:
    print(ex)

 ## List all Feature Stores

In [ ]:
list(admin_client.list_featurestores(parent=LOC_PATH))

## Get Feature store by name

In [ ]:
try:
    feature_store_obj = admin_client.get_featurestore(name = FS_PATH)
    feature_store_obj.online_serving_config.fixed_node_count=2
    admin_client.update_featurestore(
        featurestore_service_pb2.UpdateFeaturestoreRequest(featurestore=feature_store_obj)
    )
except Exception as ex:
    print(ex)
feature_store_obj.online_serving_config

### Update feature store (changing online serving nodes)

In [ ]:
feature_store_obj.online_serving_config.fixed_node_count=1
admin_client.update_featurestore(
    featurestore_service_pb2.UpdateFeaturestoreRequest(featurestore=feature_store_obj)
)

## Create a feature store Entity

In [ ]:
try:
    entity_type_obj = admin_client.create_entity_type(
        featurestore_service_pb2.CreateEntityTypeRequest(
            parent=FS_PATH,
            entity_type_id=ENTITY, 
            entity_type=entity_type_pb2.EntityType(description="planet features"))).result()
    print(entity_type_obj)
except Exception as ex:
    print(ex)

In [ ]:
type(entity_type_obj)

In [ ]:
from google.protobuf import duration_pb2 as duration 

duration = duration.Duration()
duration.FromSeconds(seconds=60)


fmc=FeaturestoreMonitoringConfig(
    snapshot_analysis = FeaturestoreMonitoringConfig.SnapshotAnalysis(disabled=False, monitoring_interval = duration))
fmc

In [ ]:
entity_type_obj.monitoring_config= fmc
entity_type_obj = admin_client.update_entity_type(featurestore_service_pb2.UpdateEntityTypeRequest(
            entity_type=entity_type_pb2.EntityType(name=entity_type_obj.name)),
            )

## Create features in batch

In [ ]:
features_list_tmp = []

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.INT64,
                    description="Mars-Sun angle",
                    monitoring_config= fmc,
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="ls"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="month ",
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="month"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.DOUBLE,
                    description="maximum temp on the day",
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="max_temp"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.DOUBLE,
                    description="minimum temp on the day",
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="min_temp"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.DOUBLE,
                    description="atmospheric pressure",
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="pressure"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.STRING,
                    description="opacity in atmosphere (i.e sunny)",
                    labels=[("no-pii", "")]),
                feature_id="atmo_opacity"))


features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.DOUBLE,
                    description="maximum temp standardized",
                    labels=[("no-pii", "")]),
                feature_id="max_temp_std"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.DOUBLE,
                    description="minimum temp standardized",
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="min_temp_std"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.DOUBLE,
                    description="atmospheric pressure standardized",
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="pressure_std"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.DOUBLE,
                    description="avarage atmospheric pressure of last 5 days",
                    labels=[("no-pii", "")]),
                feature_id="avg_pressure_5d"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.DOUBLE,
                    description="avarage min temperature of last 5 days",
                    labels=[("no-pii", ""),("approved", "")]),
                feature_id="avg_min_temp_5d"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.DOUBLE,
                    description="avarage max temperature of last 5 days",
                    labels=[("no-pii", ""),("approved", ""),("validated", "")]),
                feature_id="avg_max_temp_5d"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.DOUBLE_ARRAY,
                    description="array containing last 3 atmospheric pressures",
                    labels=[("no-pii", "")]),
                feature_id="arr_pressure_3d"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                feature=Feature(
                    value_type=Feature.ValueType.DOUBLE_ARRAY,
                    description="array containing last 3 min temperatures",
                    labels=[("no-pii", "")]),
                feature_id="arr_min_temp_3d"))

features_list_tmp.append(featurestore_service_pb2.CreateFeatureRequest(
                    feature=Feature(value_type=Feature.ValueType.DOUBLE_ARRAY,
                    description="array containing last 3 max temperatures",
                    labels=[("no-pii", "")]),
                feature_id="arr_max_temp_3d"))

try:
    features_list = admin_client.batch_create_features(
        parent=ENTITY_PATH,
        requests=features_list_tmp).result()
    print(features_list)
except Exception as ex:
    print(ex)

## Update Feature

In [ ]:
feature =admin_client.get_feature(name=FEATURE_PATH.format('atmo_opacity'))
feature

In [ ]:
tmp_list = list(feature.labels.items())
tmp_list.append(("approved", ""))
feature.labels=tmp_list
feature.monitoring_config= fmc

In [ ]:
admin_client.update_feature(
    featurestore_service_pb2.UpdateFeatureRequest(feature=feature)
)

## Delete Feature

In [ ]:
#try:
#    admin_client.delete_feature(name=FEATURE_PATH.format('last_five_max_temp')).result()
#except Exception as ex:
#    print(ex)

## Search Features

-  ``feature_id``: Supports = comparisons.
-  ``description``: Supports = comparisons. Multi-token filters should be enclosed in quotes.
-  ``entity_type_id``: Supports = comparisons.
-  ``value_type``: Supports = and != comparisons.
-  ``labels``: Supports key-value equality as well as key presence.
-  ``featurestore_id``: Supports = comparisons.

In [ ]:
admin_client.search_features(location=LOC_PATH)

In [ ]:
q="featurestore_id={}".format(FEATURESTORE_ID)
admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=LOC_PATH, query=q, page_size=2, page_token="" ))

In [ ]:
q="""
    feature_id=arr_max_temp_3d AND 
    value_type=DOUBLE_ARRAY AND 
    featurestore_id={}
""".format(FEATURESTORE_ID)

admin_client.search_features(
        featurestore_service_pb2.SearchFeaturesRequest(
            location=LOC_PATH, query=q, page_size=5))